# e) Face Recognition

### Dataset

In [160]:
import os
import pandas as pd
import numpy as np
from skimage import io
from sklearn.metrics import accuracy_score

In [161]:
def import_image(filename):
    image = io.imread('yalefaces/' +filename, as_grey = True)
    image_flat = [item for sub in image for item in sub]
    return image_flat

In [162]:
list_of_images = os.listdir('yalefaces')
data = pd.DataFrame(list_of_images)

In [163]:
data = []
for title in list_of_images:
    row = {}
    row['filename'] = title
    row['subject'] = title.split('.')[0]
    row['expression'] = title.split('.')[1]
    row['flat'] = import_image(title)
    data.append(row)
data = pd.DataFrame.from_records(data)

In [164]:
for row in range(data['flat'].count()):
    entries = len(data['flat'][0])
    numerator = 0
    for entry in data['flat'][row]:
        numerator += entry
    mean = numerator/entries
    data['flat'][row] = data['flat'][row] / mean

### Train/Test Split

In [165]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.33)
#print (train,test)

In [166]:
y_train, y_test = [], []
for ids in train['subject']:
    y_train.append(ids[7:9])
#print(y_train)
for ids in test['subject']:
    y_test.append(ids[7:9])
#print(y_test)

### i) AdaBoost: Base Estimator - DecisionTreeClassifier

In [167]:
from sklearn.ensemble import AdaBoostClassifier
A = train['flat'].values.tolist()
B = test['flat'].values.tolist()
clf = AdaBoostClassifier(n_estimators=50, learning_rate=0.1,random_state=0)
clf.fit(A,y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.1, n_estimators=50, random_state=0)

In [168]:
pred = clf.predict(B)

count = 0
for i in range(len(pred)):
    if pred[i] == y_test[i]:
        count += 1
print("AdaBoost Accuracy:", (accuracy_score(pred, y_test))*100,"%")

AdaBoost Accuracy: 65.45454545454545 %


### ii) Support Vector Machines: Kernel - Linear

In [169]:
from sklearn import svm
A = train['flat'].values.tolist()
B = test['flat'].values.tolist()
clf = svm.SVC(kernel="linear", C=100000)
clf.fit(A,y_train)

SVC(C=100000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [170]:
pred = clf.predict(B)

count = 0
for i in range(len(pred)):
    if pred[i] == y_test[i]:
        count += 1
print("SVM Accuracy:", (accuracy_score(pred, y_test))*100,"%")

SVM Accuracy: 81.81818181818183 %


# f) Pose Recognition

In [155]:
data['expression'].value_counts()

glasses        16
noglasses      15
wink           15
sad            15
leftlight      15
normal         15
happy          15
surprised      15
rightlight     15
sleepy         15
centerlight    14
gif             1
Name: expression, dtype: int64

### Replacing Categorical Poses with Hot-Encoding

In [ ]:
expression = {'glasses        ' : 0, 'noglasses      ' : 1,'wink           ' : 2,
              'sad            ' : 3,'leftlight      ' : 4, 'normal         ' : 5,'happy          ' : 6,
              'surprised      ' : 7,'rightlight     ' : 8, 'sleepy         ' : 9, 'centerlight    ' : 10}
data['expression'] = [expression[item] for item in data['expression']]

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data.expression, test_size=0.33)
#print (train,test)

In [ ]:
y_train, y_test = [], []
for ids in train:
    y_train.append(ids)
#print(y_train)
for ids in test:
    y_test.append(ids)
#print(y_test)

### Gaussian Mixture Model

In [ ]:
from sklearn.mixture import GaussianMixture
A = train.values.tolist()
B = test.values.tolist()
gmm = GaussianMixture(n_components=11).fit(A,y_train)
pred = gmm.predict(B)
count = 0
for i in range(len(pred)):
    if pred[i] == y_test[i]:
        count += 1
print("Accuracy:", (accuracy_score(pred, y_test))*100,"%")